In [30]:
#imports
from matplotlib import pyplot
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from secrets import randbelow

In [31]:
#defining functions:
#get_soup to prep for scrape
#get_reviews to scrape
#for loop inside get_reviews will select every instance of a review block and locate in it 
#Title, rating, location of reviewer and date of reviewer
#Finally, save all that to a list

def get_soup(url):
    rnum = randbelow(10)
    r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait': rnum})
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup
def get_reviews(soup):
    reviewlist = []
    reviews = soup.find_all('div',{'data-hook': 'review'})
    for item in reviews:
        try:
            review = {
            'product' : product.strip(),
            'brand' : brand.strip(),
            #'rev_title' : item.find('a', {'data-hook':'review-title'}).text.strip(),
            'rev_rating' : float(item.find('i',{'data-hook':'review-star-rating'}).text.replace('out of 5 stars','').strip()),
            'rev_loc' : re.findall(r'Reviewed in (.+) on ', item.find('span', {'data-hook':'review-date'}).text.strip())[0],
            'rev_dt' : re.findall(r' on ([0-9]{1,2} \w+ [0-9]{4})$', item.find('span', {'data-hook':'review-date'}).text.strip())[0],
            }
            reviewlist.append(review)
        except:
            pass
    return reviewlist



In [32]:
#call up list of candle products to be used for the functions
# create empty reviewlists which will be appended and added to new dataframe with reviews
# For loop to loop through each url in the candles dataset, add a page number at the end of URL
# the loop stop once it reaches the last page, which can be identified by
# list item with class "a-disabled a-last"
df_start = pd.read_excel("D:/OneDrive/Courses/Portfolio Project/Candle-19 project/candles_dataset.xlsx")
reviewlists = []
for row in df_start.itertuples():
    url = row.Links
    product = row.ProductName
    brand = row.Brand
    for x in range(1,500):
    # For some reason amazon.co.uk and amazon.com (possibly others as well) stops loading reviews past page 500
        soup = get_soup(str(url)+"&pageNumber="+str(x)+"&sortBy=recent")
        print(f'Scraping page: {x}'+' for product '+product)
        for review in get_reviews(soup):
            reviewlists.append(review)
        print(product+' reviews dataset count is now '+str(len(reviewlists))+'.')
        if not soup.find('li', {'class': 'a-disabled a-last'}):
            pass
        else:
            break

Scraping page: 1 for product All is Bright
All is Bright reviews dataset count is now 10.
Scraping page: 2 for product All is Bright
All is Bright reviews dataset count is now 20.
Scraping page: 3 for product All is Bright
All is Bright reviews dataset count is now 30.
Scraping page: 4 for product All is Bright
All is Bright reviews dataset count is now 40.
Scraping page: 5 for product All is Bright
All is Bright reviews dataset count is now 50.
Scraping page: 6 for product All is Bright
All is Bright reviews dataset count is now 60.
Scraping page: 7 for product All is Bright
All is Bright reviews dataset count is now 70.
Scraping page: 8 for product All is Bright
All is Bright reviews dataset count is now 80.
Scraping page: 9 for product All is Bright
All is Bright reviews dataset count is now 90.
Scraping page: 10 for product All is Bright
All is Bright reviews dataset count is now 100.
Scraping page: 11 for product All is Bright
All is Bright reviews dataset count is now 110.
Scrapi

In [33]:
df_reviews = pd.DataFrame(reviewlists)

In [34]:
df_reviews

,product,brand,rev_rating,rev_loc,rev_dt
0,All is Bright,Yankee Candle Tea Light Scented Candles,5.0,the United Kingdom,15 May 2022
1,All is Bright,Yankee Candle Tea Light Scented Candles,1.0,the United Kingdom,13 May 2022
2,All is Bright,Yankee Candle Tea Light Scented Candles,5.0,the United Kingdom,11 May 2022
3,All is Bright,Yankee Candle Tea Light Scented Candles,2.0,the United Kingdom,10 May 2022
4,All is Bright,Yankee Candle Tea Light Scented Candles,2.0,the United Kingdom,9 May 2022
...,...,...,...,...,...
11743,Black Peppercorn,WoodWick Ellipse Scented Candle with Crackling...,5.0,the United Kingdom,9 July 2014
11744,Black Peppercorn,WoodWick Ellipse Scented Candle with Crackling...,5.0,the United Kingdom,26 January 2014
11745,Black Peppercorn,WoodWick Ellipse Scented Candle with Crackling...,5.0,the United Kingdom,8 January 2014
11746,Black Peppercorn,WoodWick Ellipse Scented Candle with Crackling...,4.0,the United Kingdom,14 December 2013


In [35]:
#save to CSV with ; separator
df_reviews.to_csv('D:/OneDrive/Courses/Portfolio Project/Candle-19 project/tittle-less/reviews_dataset_UK.csv',sep=';') 
